## <a id="text_0"></a>Исследование надёжности заёмщиков банка

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

*В ходе работы проведем:*
1. анализ входных данных от Заказчика.
2. поиск информации, не представляющей ценности для исследования, либо не пригодной для обработки. 
3. анализ таких артефактов для предотвращения появления такой информации.
4. подготовку данных для выполнения поставленных Заказчиком задач.
5. выявление закономерности в данных для выполнения поставленной задачи банком.

### Изучение общей информации. 

In [1]:
import pandas as pd

from pymystem3 import Mystem
m = Mystem()

In [2]:
data = pd.read_csv(r'C:\Users\mind2\Downloads\Borrowers.csv') 

In [3]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,19351.0,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,4.017554e+05
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,19351.0,167422.302208,102971.566448,20667.263793,103053.152913,145017.937533,203435.067663,2.265604e+06


In [4]:
display(data.sample(5))
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
17397,1,-1031.653078,42,среднее,1,женат / замужем,0,M,сотрудник,0,105889.942646,недвижимость
20675,0,NaN,66,СРЕДНЕЕ,1,вдовец / вдова,2,F,пенсионер,0,NaN,покупка своего жилья
10594,0,-2508.110826,29,неоконченное высшее,2,женат / замужем,0,F,госслужащий,0,97000.024737,сделка с автомобилем
19583,0,-3076.317322,47,СРЕДНЕЕ,1,гражданский брак,1,F,сотрудник,0,89701.522087,свадьба
13803,0,380929.610126,71,высшее,0,в разводе,3,F,пенсионер,0,257402.642411,операции с жильем


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод 

1. В файле находятся 12 столбцов и 21525 строк. В столбцах *'days_employed'* и *'total_income'* содержатся пропуски значений, а также столбцы имеют формат float64, соответственно, числовые значения не являются читабельными. 
2. Из вышеприведенной информации можно сделать вывод, что информация, приведенная в столбце 'days_employed' ***не представляет ценности***, так как диапазон значений находится **от -50 до 1100 лет** трудовой деятельности. Ряд значений делится пополам на значении -3,3 года, а это означает, что большая часть значений со знаком '-', а среднее значение на уровне 156 лет. Появление такого массива данных тоже можно назвать системной ошибкой. Большое количество значений со знаком минус, значит, происходит процесс вычитания. Скорее всего, данные получены за счет не корректного вычитанния даты начала работы, например в EXCEL.
3. В столбце *'children'* минимальное значение равно '-1', а максимальное значение равно '20'. Таких значений быть не может исходя из анализа процентиля.
4. В столбце *'dob_years'* минимальное значение возраста является '0'. Такого значения быть не может.
5. Исходя из анализа процентиля столбца *'total_income'* установим градацию:
   - менее 100 000
   - от 100 000 до 150 000
   - от 150 000 до 200 000
   - более 200 000.

### Предобработка данных

### <a id="text_3"></a> Обработка пропусков

Проведем подсчет количества пропусков в столбцах:

In [5]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Как и показывала общая информация, в столбцах 'days_employed' и 'total_income' содержатся по 2174 пропусков. 

Так как в столбцах 'days_employed' и 'total_income' представлены числовые значения, то проведем замену методом .fillna() на значение '0' для более удобного проведения дальнейшего расчета.

In [6]:
data['days_employed']=data['days_employed'].fillna(0)
data['total_income']=data['total_income'].fillna(0)

Проведем проверку количества пропусков в таблице:

In [7]:
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Проверка показала отсутствие пропусков в таблице.

### Вывод

Выявили пропуски значений данных в файле, провели замену методом .fillna(). Такие системные пропуски могли произойти из-за не предоставления информации заемщиков банку, например, не предоставление справки с места работы или справки о доходах. 
Из вывода общей информации видно, что в столбцах 'days_employed' и 'total_income' тип данных соответствует 'float64'.

### <a id="text_4"></a> Замена типа данных

Проведем замену типа данных в столбцах 'days_employed' и 'total_income' на тип данных 'int':

In [8]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

Применим функцию Type_check для проверки правильности замены типа данных на 'int':

In [9]:
def Type_check(column):
    total_position = 0
    count_lines = 0
    wrong_lines_content = []
    for row in data[column]:
        try:
            count_lines += 1
            level = int(row)
            total_position += level
        except:
            wrong_lines_content.append(row)
    print('Количество измерений',count_lines)
    print('Некорректные строки',wrong_lines_content)

Type_check('days_employed')
Type_check('total_income')

Количество измерений 21525
Некорректные строки []
Количество измерений 21525
Некорректные строки []


Проведем обработку столбца **'children'**

In [10]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

   Из приведенной выше таблице можно увидеть 2 артефакта: 
1. Значение 20 явно выделяется и не может иметь показатель 76 больше, чем у значений 4 и 5 вместе взятых. 
2. Значение -1 тоже не может относиться к действительности.
   Заменим артефакты на новые значения и проверим результат.

In [11]:
data['children'] = data['children'].replace(-1,1)
data['children'] = data['children'].replace(20,2)
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Проведем обработку столбца **'dob_years'**

In [12]:
data.groupby('dob_years').count()

,children,days_employed,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
dob_years,,,,,,,,,,,
0,101,101,101,101,101,101,101,101,101,101,101
19,14,14,14,14,14,14,14,14,14,14,14
20,51,51,51,51,51,51,51,51,51,51,51
21,111,111,111,111,111,111,111,111,111,111,111
22,183,183,183,183,183,183,183,183,183,183,183
23,254,254,254,254,254,254,254,254,254,254,254
24,264,264,264,264,264,264,264,264,264,264,264
25,357,357,357,357,357,357,357,357,357,357,357
26,408,408,408,408,408,408,408,408,408,408,408


Из таблицы видно, что аномальным является значение возраста "0". Заменим на выпавшее значение "18".

In [13]:
data['dob_years'] = data['dob_years'].replace(0,18)

Также проверим обработаем столбцы **'education_id', 'family_status_id', 'debt'**

In [14]:
data.groupby('education_id').count()

,children,days_employed,dob_years,education,family_status,family_status_id,gender,income_type,debt,total_income,purpose
education_id,,,,,,,,,,,
0,5260,5260,5260,5260,5260,5260,5260,5260,5260,5260,5260
1,15233,15233,15233,15233,15233,15233,15233,15233,15233,15233,15233
2,744,744,744,744,744,744,744,744,744,744,744
3,282,282,282,282,282,282,282,282,282,282,282
4,6,6,6,6,6,6,6,6,6,6,6


В столбце **'education_id'** артефактов не выявлено.

In [15]:
data.groupby('family_status_id').count()

,children,days_employed,dob_years,education,education_id,family_status,gender,income_type,debt,total_income,purpose
family_status_id,,,,,,,,,,,
0,12380,12380,12380,12380,12380,12380,12380,12380,12380,12380,12380
1,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177
2,960,960,960,960,960,960,960,960,960,960,960
3,1195,1195,1195,1195,1195,1195,1195,1195,1195,1195,1195
4,2813,2813,2813,2813,2813,2813,2813,2813,2813,2813,2813


В столбце **'family_status_id'** артефактов не выявлено.

In [16]:
data.groupby('debt').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,total_income,purpose
debt,,,,,,,,,,,
0,19784,19784,19784,19784,19784,19784,19784,19784,19784,19784,19784
1,1741,1741,1741,1741,1741,1741,1741,1741,1741,1741,1741


В столбце **'debt'** артефактов не выявлено.

Возьмем функцию из [статьи](https://habr.com/ru/company/ruvds/blog/442516/) `mem_usage` для подсчета используемой памяти до и после преобразования, а также выведем оптимальные типы численных значений столбцов:

In [17]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: 
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 
    return "{:0.2f} MB".format(usage_mb)

data_int = data.select_dtypes(include=['int'])
converted_int = data_int.apply(pd.to_numeric,downcast='unsigned')

print(mem_usage(data_int))
print(mem_usage(converted_int))

compare_ints = pd.concat([data_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

0.16 MB
0.16 MB


,before,after
int32,2.0,1
uint32,NaN,1


Из приведенной выше информации видно, что преобразования уменьшат обьем используемой памяти почти на 70% для численных значений данных.

Проведем преобразования типов данных:

In [18]:
data['dob_years'] = data['dob_years'].astype('uint8')
data['children'] = data['children'].astype('uint8')
data['education_id'] = data['education_id'].astype('uint8')
data['family_status_id'] = data['family_status_id'].astype('uint8')
data['debt'] = data['debt'].astype('uint8')
data['total_income'] = data['total_income'].astype('uint32')

Далее используем алгоритм [из статьи](https://habr.com/ru/company/ruvds/blog/442516/) преобразования данных типа **'object'** для уменьшения используемой памяти.

Для начала выведем анализ столбцов с данными типа **'object'**:

In [19]:
data_obj = data.select_dtypes(include=['object']).copy()
data_obj.describe()

,education,family_status,gender,income_type,purpose
count,21525,21525,21525,21525,21525
unique,15,5,3,8,38
top,среднее,женат / замужем,F,сотрудник,свадьба
freq,13750,12380,14236,11119,797


Далее проанализируем уменьшение объема памяти при проведении показателей:

In [20]:
converted_obj = pd.DataFrame()

for col in data_obj.columns:
    num_unique_values = len(data_obj[col].unique())
    num_total_values = len(data_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = data_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = data_obj[col]

In [21]:
print(mem_usage(data_obj))
print(mem_usage(converted_obj))

compare_obj = pd.concat([data_obj.dtypes,converted_obj.dtypes],axis=1)
compare_obj.columns = ['before','after']
compare_obj.apply(pd.Series.value_counts)

12.34 MB
0.11 MB


,before,after
object,5.0,NaN
category,NaN,1.0
category,NaN,1.0
category,NaN,1.0
category,NaN,1.0
category,NaN,1.0


После проведения преобразований размер памяти уменьшится на 99%. Проведем преобразования:

In [22]:
data['education'] = data['education'].astype('category')
data['family_status'] = data['family_status'].astype('category')
data['gender'] = data['gender'].astype('category')
data['income_type'] = data['income_type'].astype('category')
data['purpose'] = data['purpose'].astype('category')

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   children          21525 non-null  uint8   
 1   days_employed     21525 non-null  int32   
 2   dob_years         21525 non-null  uint8   
 3   education         21525 non-null  category
 4   education_id      21525 non-null  uint8   
 5   family_status     21525 non-null  category
 6   family_status_id  21525 non-null  uint8   
 7   gender            21525 non-null  category
 8   income_type       21525 non-null  category
 9   debt              21525 non-null  uint8   
 10  total_income      21525 non-null  uint32  
 11  purpose           21525 non-null  category
dtypes: category(5), int32(1), uint32(1), uint8(5)
memory usage: 381.5 KB


### Вывод

Провели анализ занимаемого информацией объема памяти. После проведения замены типа данных с численными значениями столбцов получили уменьшение памяти на 70% ***(с 1.15 MB до 0,35 MB)***. Провели замену типа данных в столбцах с  **'object'** на **'category'**. Уменьшение занимаемой памяти составило 99% ***(с 12.34 MB до 0,11 MB)***. В итоге уменьшение объема занимаемой памяти ***составило 97%*** без потери важной информации для работы.

### <a id="text_5"></a> Обработка дубликатов

Проведем обработку данных от дублирующей информации

Сперва уберем все заглавные буквы из столбцов с типом данных 'object'

In [24]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['gender'] = data['gender'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['purpose'] = data['purpose'].str.lower()

Посчитаем количество дублирующих строк 

In [25]:
data.duplicated().sum()

71

Уберем дублирующие строки

In [26]:
data = data.drop_duplicates().reset_index(drop = True)

Проверим наличие дублирующих строк.

In [27]:
data.duplicated().sum()

0

In [28]:
data.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
17609,3,-4761,44,среднее,1,женат / замужем,0,f,сотрудник,0,227701,получение образования
12900,2,-2406,32,высшее,0,женат / замужем,0,f,сотрудник,0,121616,недвижимость
7965,0,401345,66,среднее,1,вдовец / вдова,2,f,пенсионер,0,85782,на покупку своего автомобиля
21404,0,365213,52,среднее,1,женат / замужем,0,f,пенсионер,1,48239,свой автомобиль
18125,0,0,53,среднее,1,женат / замужем,0,f,пенсионер,1,0,покупка своего жилья


# Вывод

Провели обработку методом **drop_duplicates().reset_index 71 дублирующей строки**. Дублирующие строчки появляются при внесении в базу заемщиков, либо полное совпадение из-за отсутствия уникальных идентификаторов. Для поиска дубликатов были изменены заглавные буквы на строчные. 

### <a id="text_6"></a> Лемматизация

Проведем лемматизацию столбцов **'education', 'family_status', 'gender','income_type', 'purpose'** при помощи функции Clemmas. Выведем значения в столбцах.

In [29]:
data_ed = data['education'].unique()
data_fam = data['family_status'].unique()
data_gen = data['gender'].unique()
data_inty = data['income_type'].unique()
data_pur = data['purpose'].unique()

def Clemmas(infor):
    i=0
    Clemas = []
    for i in range(len(infor)):
        lemmas = m.lemmatize(infor[i])
        Clemas.append(lemmas)
    return print(Clemas)

Проведем лемматизацию столбца ***'education'***.

In [30]:
Clemmas(data_ed)

[['высокий', '\n'], ['средний', '\n'], ['неоконченный', ' ', 'высокий', '\n'], ['начальный', '\n'], ['ученый', ' ', 'степень', '\n']]


Дальнейшей обработке столбец не подлежит. Проведем лемматизацию столбца ***'family_status'***.

In [31]:
Clemmas(data_fam)

[['женатый', ' / ', 'замужем', '\n'], ['гражданский', ' ', 'брак', '\n'], ['вдовец', ' / ', 'вдова', '\n'], ['в', ' ', 'развод', '\n'], ['не', ' ', 'женатый', ' / ', 'не', ' ', 'замужем', '\n']]


Также столбец обработке не подлежит. Проведем лемматизацию столбца ***'gender'***.

In [32]:
Clemmas(data_gen)

[['f', '\n'], ['m', '\n'], ['xna', '\n']]


Проведем исследованние значения ***'xna'***.

In [33]:
data.loc[data.loc[:,'gender'] == 'xna']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10684,0,-2358,24,неоконченное высшее,2,гражданский брак,1,xna,компаньон,0,203905,покупка недвижимости


In [34]:
data['gender'].value_counts()

f      14174
m       7279
xna        1
Name: gender, dtype: int64

В данных встречается только одна строчка со значением **'xna'** в столбце **'gender'**. Все данные в строке 10684 носят реальный характер. Присоединим это значение к значению 'm'. Изменение в количественном составе значения 'm' составят ***менее 0,02 %***.

In [35]:
data = data.replace('xna','m')

Проведем лемматизацию столбца **'income_type'**.

In [36]:
Clemmas(data_inty)

[['сотрудник', '\n'], ['пенсионер', '\n'], ['компаньон', '\n'], ['госслужащий', '\n'], ['безработный', '\n'], ['предприниматель', '\n'], ['студент', '\n'], ['в', ' ', 'декрет', '\n']]


Компаньон - это совладелец организации. Объединим вместе с предпринимателем. 

In [37]:
data = data.replace('компаньон','предприниматель')
data['income_type'].value_counts()

сотрудник          11084
предприниматель     5080
пенсионер           3829
госслужащий         1457
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

Проведем лемматизацию столбца **'purpose'**.

In [38]:
Clemmas(data_pur)

[['покупка', ' ', 'жилье', '\n'], ['приобретение', ' ', 'автомобиль', '\n'], ['дополнительный', ' ', 'образование', '\n'], ['сыграть', ' ', 'свадьба', '\n'], ['операция', ' ', 'с', ' ', 'жилье', '\n'], ['образование', '\n'], ['на', ' ', 'проведение', ' ', 'свадьба', '\n'], ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'], ['покупка', ' ', 'недвижимость', '\n'], ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'], ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'], ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'], ['недвижимость', '\n'], ['строительство', ' ', 'недвижимость', '\n'], ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'], ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'], ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'], ['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n'], ['жилье', '\n'], ['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n'], ['автомобиль', '\n'], ['заниматьс

Проведем уменьшение целей получения кредита. Останутся следующие цели: **свадьба, автомобиль, образование, недвижимость, жилье**.

In [40]:
list_of_purpose = {'свад':1, 'авто':2, 'образ':3, 'недв':4, 'жил':5}
data['purpose_category'] = 0

for row in range(len(data)):
    for purpose in list_of_purpose:
        if purpose in data.iloc[row, 11]:
            data.iloc[row, 12] = list_of_purpose[purpose]
        
data.sample(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
5488,2,-4937,37,среднее,1,женат / замужем,0,m,сотрудник,0,94184,недвижимость,4
21318,0,-441,36,среднее,1,женат / замужем,0,f,сотрудник,0,113788,сделка с подержанным автомобилем,2
13389,0,-947,18,высшее,0,женат / замужем,0,m,предприниматель,0,182747,на покупку своего автомобиля,2
7549,0,-1706,32,среднее,1,женат / замужем,0,f,сотрудник,0,65419,недвижимость,4
339,1,-1561,32,среднее,1,женат / замужем,0,f,сотрудник,0,79756,сделка с подержанным автомобилем,2
18202,1,379268,57,среднее,1,женат / замужем,0,m,пенсионер,0,54253,получение образования,3
16777,1,0,29,среднее,1,женат / замужем,0,f,госслужащий,0,0,жилье,5
4571,1,-1619,25,среднее,1,женат / замужем,0,f,сотрудник,0,130218,заняться высшим образованием,3
17907,0,-3165,55,высшее,0,не женат / не замужем,4,f,предприниматель,0,170851,автомобили,2
220,1,0,23,неоконченное высшее,2,гражданский брак,1,f,предприниматель,0,0,сыграть свадьбу,1


In [41]:
data = data.replace('на проведение свадьбы','свадьба')
data = data.replace('сыграть свадьбу','свадьба')

data = data.replace('сделка с подержанным автомобилем','автомобиль')
data = data.replace('свой автомобиль','автомобиль')
data = data.replace('автомобили','автомобиль')
data = data.replace('на покупку подержанного автомобиля','автомобиль')
data = data.replace('на покупку своего автомобиля','автомобиль')
data = data.replace('на покупку автомобиля','автомобиль')
data = data.replace('приобретение автомобиля','автомобиль')
data = data.replace('сделка с автомобилем','автомобиль')

data = data.replace('получение высшего образования','образование')
data = data.replace('заняться образованием','образование')
data = data.replace('профильное образование','образование')
data = data.replace('получение образования','образование')
data = data.replace('получение дополнительного образования','образование')
data = data.replace('высшее образование','образование')
data = data.replace('дополнительное образование','образование')
data = data.replace('заняться высшим образованием','образование')

data = data.replace('операции с недвижимостью','недвижимость')
data = data.replace('покупка коммерческой недвижимости','недвижимость')
data = data.replace('строительство жилой недвижимости','недвижимость')
data = data.replace('операции с коммерческой недвижимостью','недвижимость')
data = data.replace('строительство собственной недвижимости','недвижимость')
data = data.replace('покупка жилой недвижимости','недвижимость')
data = data.replace('операции со своей недвижимостью','недвижимость')
data = data.replace('строительство недвижимости','недвижимость')
data = data.replace('покупка недвижимости','недвижимость')

data = data.replace('покупка жилья для сдачи','жилье')
data = data.replace('покупка своего жилья','жилье')
data = data.replace('покупка жилья','жилье')
data = data.replace('ремонт жилью','жилье')
data = data.replace('покупка жилья для семьи','жилье')
data = data.replace('операции с жильем','жилье')

In [42]:
data['purpose'].value_counts()

недвижимость    6351
жилье           4460
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose, dtype: int64

### Вывод

Провели лемматизацию столбцов **'education', 'family_status', 'gender','income_type', 'purpose'**. Объединили значение 'xna' со значением 'm' в столбце 'gender'. Также объединили 'компаньон' и 'предприниматель' в столбце 'income_type'. ***Уменьшили количество целей получения кредита с 38 до 5.***

### <a id="text_7"></a> Категоризация данных

Проведем категоризацию данных для следующих столбцов: **'children', 'dob_years', 'total_income'**.

Проведем категоризацию для количества детей у заемщиков. Разделим заемщиков на следующие категории: 
1. Заемщики не имеют детей.
2. Заемщики имеют от 1 до 2 детей.
3. Заемщики имеют 3 и более детей.

In [43]:
def child_group(child):
               if child == 0:
                    return 'не имеют детей'
               if 1 <= child <= 2:
                    return 'есть дети'
               if child > 2:
                    return 'много детей'
data['child_group'] = data['children'].apply(child_group)
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,child_group
0,1,-8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,жилье,5,есть дети
1,1,-4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,автомобиль,2,есть дети
2,0,-5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,жилье,5,не имеют детей
3,3,-4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,образование,3,много детей
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,свадьба,1,не имеют детей
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-4529,43,среднее,1,гражданский брак,1,f,предприниматель,0,224791,жилье,5,есть дети
21450,0,343937,67,среднее,1,женат / замужем,0,f,пенсионер,0,155999,автомобиль,2,не имеют детей
21451,1,-2113,38,среднее,1,гражданский брак,1,m,сотрудник,1,89672,недвижимость,4,есть дети
21452,3,-3112,38,среднее,1,женат / замужем,0,m,сотрудник,1,244093,автомобиль,2,много детей


Получили категоризацию по количеству детей. Информация отображается в 13 столбце.

Проведем категоризацию по возрасту заемщика. Делим на три категории:
1. Лица, моложе 23 лет (учащиеся и студенты).
2. Лица работоспособного возраста от 24 до 60 лет.
3. Лица пенсионного возраста от 60 лет.

In [44]:
def age_level(age):
               if age <= 23:
                    return 'молодой'
               if 23 <= age <= 60:
                    return 'работоспособный'
               if      age > 60:
                    return 'пожилой'
data['age_level'] = data['dob_years'].apply(age_level)
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,child_group,age_level
0,1,-8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,жилье,5,есть дети,работоспособный
1,1,-4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,автомобиль,2,есть дети,работоспособный
2,0,-5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,жилье,5,не имеют детей,работоспособный
3,3,-4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,образование,3,много детей,работоспособный
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,свадьба,1,не имеют детей,работоспособный
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-4529,43,среднее,1,гражданский брак,1,f,предприниматель,0,224791,жилье,5,есть дети,работоспособный
21450,0,343937,67,среднее,1,женат / замужем,0,f,пенсионер,0,155999,автомобиль,2,не имеют детей,пожилой
21451,1,-2113,38,среднее,1,гражданский брак,1,m,сотрудник,1,89672,недвижимость,4,есть дети,работоспособный
21452,3,-3112,38,среднее,1,женат / замужем,0,m,сотрудник,1,244093,автомобиль,2,много детей,работоспособный


Получили категоризацию по возрасту.
Далее, для категоризации по ежемесячному доходу, проведем усреднение и заполнение пропущенных значений в столбце 'income_type'. Проведем категоризацию по столбцам 'age_level' и 'income_type'. Считаем, что на ежемесячный доход влияет возраст как показатель опыта и должность как показатель знаний и образования.

In [45]:
total_income_zero = data[data['total_income'] == 0]
total_income_zero.groupby('age_level')['debt'].count()

age_level
молодой              85
пожилой             209
работоспособный    1809
Name: debt, dtype: int64

В строках для значения столбца 'total_income' = 0 присутствует вышеуказанная возрастная категоризация. 

In [46]:
total_income_zero.groupby('income_type')['debt'].count()

income_type
госслужащий         145
пенсионер           386
предприниматель     502
сотрудник          1070
Name: debt, dtype: int64

Вышеуказанные профессии находятся в строках для значения столбца 'total_income' = 0. Проведем категоризацию по значениям этих двух столбцов:

In [47]:
def ageinc_type(row):
    
    age_level = row['age_level']
    income_type = row['income_type']
    
    if (age_level == 'работоспособный') & (income_type == 'сотрудник'):
        return 'работоспособный сотрудник'
    if (age_level == 'работоспособный') & (income_type == 'предприниматель'):
        return 'работоспособный предприниматель'
    if (age_level == 'работоспособный') & (income_type == 'пенсионер'):
        return 'работоспособный пенсионер'
    if (age_level == 'работоспособный') & (income_type == 'госслужащий'):
        return 'работоспособный госслужащий'
    if (age_level == 'пожилой') & (income_type == 'сотрудник'):
        return 'пожилой сотрудник'
    if (age_level == 'пожилой') & (income_type == 'предприниматель'):
        return 'пожилой предприниматель'
    if (age_level == 'пожилой') & (income_type == 'пенсионер'):
        return 'пожилой пенсионер'
    if (age_level == 'пожилой') & (income_type == 'госслужащий'):
        return 'пожилой госслужащий'
    if (age_level == 'молодой') & (income_type == 'сотрудник'):
        return 'молодой сотрудник'
    if (age_level == 'молодой') & (income_type == 'предприниматель'):
        return 'молодой предприниматель'
    if (age_level == 'молодой') & (income_type == 'пенсионер'):
        return 'молодой пенсионер'
    if (age_level == 'молодой') & (income_type == 'госслужащий'):
        return 'молодой госслужащий'
               
data['status'] = data.apply(ageinc_type, axis=1)
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,child_group,age_level,status
0,1,-8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,жилье,5,есть дети,работоспособный,работоспособный сотрудник
1,1,-4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,автомобиль,2,есть дети,работоспособный,работоспособный сотрудник
2,0,-5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,жилье,5,не имеют детей,работоспособный,работоспособный сотрудник
3,3,-4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,образование,3,много детей,работоспособный,работоспособный сотрудник
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,свадьба,1,не имеют детей,работоспособный,работоспособный пенсионер
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-4529,43,среднее,1,гражданский брак,1,f,предприниматель,0,224791,жилье,5,есть дети,работоспособный,работоспособный предприниматель
21450,0,343937,67,среднее,1,женат / замужем,0,f,пенсионер,0,155999,автомобиль,2,не имеют детей,пожилой,пожилой пенсионер
21451,1,-2113,38,среднее,1,гражданский брак,1,m,сотрудник,1,89672,недвижимость,4,есть дети,работоспособный,работоспособный сотрудник
21452,3,-3112,38,среднее,1,женат / замужем,0,m,сотрудник,1,244093,автомобиль,2,много детей,работоспособный,работоспособный сотрудник


Получили категоризацию по двум столбцам. Далее разделим данные data на данные, содержащиеся в строках, где значения столбца 'total_income' = 0 и где больше 0. 

In [48]:
total_income_zero = data[data['total_income'] == 0]
total_income_full = data[data['total_income'] > 0]
total_income_zero['status'].value_counts()

работоспособный сотрудник          973
работоспособный предприниматель    473
работоспособный пенсионер          225
пожилой пенсионер                  158
работоспособный госслужащий        138
молодой сотрудник                   59
пожилой сотрудник                   38
молодой предприниматель             21
пожилой предприниматель              8
пожилой госслужащий                  5
молодой пенсионер                    3
молодой госслужащий                  2
Name: status, dtype: int64

Получили количество значений столбца 'status'. Посчитаем средние значения ежемесячного дохода для столбца 'status'.

In [49]:
total_full = total_income_full.groupby('status')['total_income'].mean()
total_full

status
молодой госслужащий                138062.290323
молодой пенсионер                  135763.222222
молодой предприниматель            155115.744681
молодой сотрудник                  134712.976864
пожилой госслужащий                188917.347826
пожилой пенсионер                  133609.955293
пожилой предприниматель            203126.121951
пожилой сотрудник                  173412.295154
работоспособный госслужащий        171050.835628
работоспособный пенсионер          139949.412290
работоспособный предприниматель    204550.118116
работоспособный сотрудник          162192.906895
Name: total_income, dtype: float64

Присвоим средние значения 'total_income' по категориям столбца 'status' значениям 'total_income' датасета 'total_income_zero'.

In [50]:
total_in0 = total_income_zero.loc[total_income_zero.loc[:,'status'] == 'молодой госслужащий']
total_in0.loc[:,'total_income'] = total_full[0]

total_in1 = total_income_zero.loc[total_income_zero.loc[:,'status'] == 'молодой пенсионер']
total_in1.loc[:,'total_income'] = total_full[1]

total_in2 = total_income_zero.loc[total_income_zero.loc[:,'status'] == 'молодой предприниматель']
total_in2.loc[:,'total_income'] = total_full[2]

total_in3 = total_income_zero.loc[total_income_zero.loc[:,'status'] == 'молодой сотрудник']
total_in3.loc[:,'total_income'] = total_full[3]

total_in4 = total_income_zero.loc[total_income_zero.loc[:,'status'] == 'пожилой госслужащий']
total_in4.loc[:,'total_income'] = total_full[4]

total_in5 = total_income_zero.loc[total_income_zero.loc[:,'status'] == 'пожилой пенсионер']
total_in5.loc[:,'total_income'] = total_full[5]

total_in6 = total_income_zero.loc[total_income_zero.loc[:,'status'] == 'пожилой предприниматель']
total_in6.loc[:,'total_income'] = total_full[6]

total_in7 = total_income_zero.loc[total_income_zero.loc[:,'status'] == 'пожилой сотрудник']
total_in7.loc[:,'total_income'] = total_full[7]

total_in8 = total_income_zero.loc[total_income_zero.loc[:,'status'] == 'работоспособный госслужащий']
total_in8.loc[:,'total_income'] = total_full[8]

total_in9 = total_income_zero.loc[total_income_zero.loc[:,'status'] == 'работоспособный пенсионер']
total_in9.loc[:,'total_income'] = total_full[9]

total_in10 = total_income_zero.loc[total_income_zero.loc[:,'status'] == 'работоспособный предприниматель']
total_in10.loc[:,'total_income'] = total_full[10]

total_in11 = total_income_zero.loc[total_income_zero.loc[:,'status'] == 'работоспособный сотрудник']
total_in11.loc[:,'total_income'] = total_full[11]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Проведем собирание датасета командой concat и присвоим столбцу 'total_income' целочисленный тип данных.

In [51]:
data_new = pd.concat([total_income_full,total_in0,total_in1,total_in2,total_in3,total_in4,total_in5,total_in6,total_in7,total_in8,total_in9,total_in10,total_in11])

data_new['total_income'] = data_new['total_income'].astype('uint32')
data_new

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,child_group,age_level,status
0,1,-8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,жилье,5,есть дети,работоспособный,работоспособный сотрудник
1,1,-4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,автомобиль,2,есть дети,работоспособный,работоспособный сотрудник
2,0,-5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,жилье,5,не имеют детей,работоспособный,работоспособный сотрудник
3,3,-4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,образование,3,много детей,работоспособный,работоспособный сотрудник
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,свадьба,1,не имеют детей,работоспособный,работоспособный пенсионер
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21361,1,0,38,неоконченное высшее,2,не женат / не замужем,4,f,сотрудник,0,162192,жилье,5,есть дети,работоспособный,работоспособный сотрудник
21392,1,0,35,высшее,0,гражданский брак,1,m,сотрудник,0,162192,свадьба,1,есть дети,работоспособный,работоспособный сотрудник
21424,1,0,50,среднее,1,гражданский брак,1,f,сотрудник,0,162192,свадьба,1,есть дети,работоспособный,работоспособный сотрудник
21431,1,0,42,среднее,1,женат / замужем,0,f,сотрудник,0,162192,недвижимость,5,есть дети,работоспособный,работоспособный сотрудник


Далее проведем категоризацию по уровню дохода.

In [52]:
def income_level(money):
               if money <= 100000:
                    return 'низкий уровень дохода'
               if 100000 <= money <= 150000:
                    return 'средний уровень дохода'
               if 150000 <= money <= 200000:
                    return 'доход выше среднего'
               if           money > 200000:
                    return 'обеспеченный уровень дохода'
data_new['money_level_group'] = data_new['total_income'].apply(income_level)
data_new

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,child_group,age_level,status,money_level_group
0,1,-8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,жилье,5,есть дети,работоспособный,работоспособный сотрудник,обеспеченный уровень дохода
1,1,-4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,автомобиль,2,есть дети,работоспособный,работоспособный сотрудник,средний уровень дохода
2,0,-5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,жилье,5,не имеют детей,работоспособный,работоспособный сотрудник,средний уровень дохода
3,3,-4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,образование,3,много детей,работоспособный,работоспособный сотрудник,обеспеченный уровень дохода
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,свадьба,1,не имеют детей,работоспособный,работоспособный пенсионер,доход выше среднего
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21361,1,0,38,неоконченное высшее,2,не женат / не замужем,4,f,сотрудник,0,162192,жилье,5,есть дети,работоспособный,работоспособный сотрудник,доход выше среднего
21392,1,0,35,высшее,0,гражданский брак,1,m,сотрудник,0,162192,свадьба,1,есть дети,работоспособный,работоспособный сотрудник,доход выше среднего
21424,1,0,50,среднее,1,гражданский брак,1,f,сотрудник,0,162192,свадьба,1,есть дети,работоспособный,работоспособный сотрудник,доход выше среднего
21431,1,0,42,среднее,1,женат / замужем,0,f,сотрудник,0,162192,недвижимость,5,есть дети,работоспособный,работоспособный сотрудник,доход выше среднего


### Вывод

In [53]:
data_new['money_level_group'].value_counts()

средний уровень дохода         6151
обеспеченный уровень дохода    5547
доход выше среднего            5293
низкий уровень дохода          4463
Name: money_level_group, dtype: int64

Провели категоризацию данных для следующих столбцов: **'children', 'dob_years', 'total_income'**. Получили категоризованные данные в столбцах: *'child_group', 'age_level', 'status', 'money_level_group'*. Усреднили по категориям уровень доходов за месяц и заменили отсутствующие значения. Эти действия позволили сохранить информацию о 10 % заемщиков. Подготовили датасет для ответа на основные вопросы исследования.

### Ответы на вопросы

**<a id="text_8"></a> - Есть ли зависимость между наличием детей и возвратом кредита в срок?**

Проведем группировку данных методом groupby с суммированием значений столбца 'debt'

In [54]:
datach_sum = data_new.groupby('child_group')['debt'].sum()
datach_sum

child_group
есть дети          647.0
много детей         31.0
не имеют детей    1063.0
Name: debt, dtype: float64

Посчитаем количество заемщиков, категоризированных по количеству детей

In [55]:
datach_count = data_new.groupby('child_group')['debt'].count()
datach_count 

child_group
есть дети          6983
много детей         380
не имеют детей    14091
Name: debt, dtype: int64

Определим отношение количества задолженностей к количеству заемщиков, категоризированных по количеству детей.

In [56]:
datach = datach_sum / datach_count
datach.sort_values()

child_group
не имеют детей    0.075438
много детей       0.081579
есть дети         0.092654
Name: debt, dtype: float64

### Вывод

Заемщики, не имеющие детей, больше всего имели задолжностей по кредитам, но и количество заемщиков значительно превышает другие категории. В итоге, если брать удельную задолжность группы, то получается, что **заемщики без детей более дисциплинировано выплачивают кредиты**. 

**<a id="text_9"></a>- Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Проведем группировку данных методом groupby с суммированием значений столбца 'debt'

In [57]:
datafam_sum = data_new.groupby('family_status')['debt'].sum()
datafam_sum

family_status
в разводе                 85.0
вдовец / вдова            63.0
гражданский брак         388.0
женат / замужем          931.0
не женат / не замужем    274.0
Name: debt, dtype: float64

Посчитаем количество заемщиков, категоризированных по семейному положению

In [58]:
datafam_count = data_new.groupby('family_status')['debt'].count()
datafam_count

family_status
в разводе                 1195
вдовец / вдова             959
гражданский брак          4151
женат / замужем          12339
не женат / не замужем     2810
Name: debt, dtype: int64

Определим отношение количества задолженностей к количеству заемщиков, категоризированных по семейному положению.

In [59]:
datafam = datafam_sum / datafam_count
datafam.sort_values()

family_status
вдовец / вдова           0.065693
в разводе                0.071130
женат / замужем          0.075452
гражданский брак         0.093471
не женат / не замужем    0.097509
Name: debt, dtype: float64

### Вывод

Исходя из полученных данных, можно сделать вывод, что вдовы/вдовцы или заемщики в разводе лучше выплачивают кредиты, чем не женатые/не замужние, либо живущие в гражданском браке.

**<a id="text_10"></a>- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Проведем группировку данных методом groupby с суммированием значений столбца 'debt'

In [60]:
datatot_sum = data_new.groupby('money_level_group')['debt'].sum()
datatot_sum

money_level_group
доход выше среднего            470.0
низкий уровень дохода          354.0
обеспеченный уровень дохода    386.0
средний уровень дохода         531.0
Name: debt, dtype: float64

Посчитаем количество заемщиков, категоризированных по уровню доходов

In [61]:
datatot_count = data_new.groupby('money_level_group')['debt'].count()
datatot_count

money_level_group
доход выше среднего            5293
низкий уровень дохода          4463
обеспеченный уровень дохода    5547
средний уровень дохода         6151
Name: debt, dtype: int64

Определим отношение количества задолженностей к количеству заемщиков, категоризированных по уровню доходов.

In [62]:
datatot = datatot_sum / datatot_count
datatot.sort_values()

money_level_group
обеспеченный уровень дохода    0.069587
низкий уровень дохода          0.079319
средний уровень дохода         0.086327
доход выше среднего            0.088797
Name: debt, dtype: float64

### Вывод

Как показали расчеты, заемщики, имеющие ежемесячный доход более 200 000, выплачивают лучше, чем заемщики из других категорий. Парадоксально, но заемщики с доходом от 100 000 до 200 000 выплачивают кредиты гораздо хуже. 

**<a id="text_11"></a>- Как разные цели кредита влияют на его возврат в срок?**

Проведем группировку данных методом groupby с суммированием значений столбца 'debt'

In [63]:
datapur_sum = data_new.groupby('purpose')['debt'].sum()
datapur_sum

purpose
автомобиль      403.0
жилье           308.0
недвижимость    474.0
образование     370.0
свадьба         186.0
Name: debt, dtype: float64

Посчитаем количество заемщиков с определенными целями получения кредита 

In [64]:
datapur_count = data_new.groupby('purpose')['debt'].count()
datapur_count

purpose
автомобиль      4306
жилье           4460
недвижимость    6351
образование     4013
свадьба         2324
Name: debt, dtype: int64

Определим отношение количества задолженностей к количеству заемщиков с определенными целями получения кредита

In [65]:
datapur = datapur_sum / datapur_count
datapur.sort_values()

purpose
жилье           0.069058
недвижимость    0.074634
свадьба         0.080034
образование     0.092200
автомобиль      0.093590
Name: debt, dtype: float64

### Вывод

Получается, что ипотека,особенно связанная с покупкой жилого помещения, для банка гораздо безопаснее, чем кредиты на образование и покупку автомобиля.

###  Общий вывод

Анализируя данные о заемщиках банка выявили следующую информацию:

1. Файл представляет из себя датасет из 12 столбцов и 21525 строк. 
2. В столбцах *'days_employed'* и *'total_income'* содержатся пропуски значений, а также столбцы имеют формат float64, соответственно, числовые значения не являются читабельными. Такие системные пропуски могли произойти из-за не предоставления информации заемщиков банку, например, не предоставление справки с места работы или справки о доходах. Усреднили по категориям уровень доходов за месяц и заменили отсутствующие значения. Эти действия позволили сохранить информацию о **10 % заемщиков**.Исходя из анализа процентиля столбца *'total_income'* установим градацию:
   - менее 100 000
   - от 100 000 до 150 000
   - от 150 000 до 200 000
   - более 200 000.
3. Информация, приведенная в столбце 'days_employed' ***не представляет ценности***, так как диапазон значений находится **от -50 до 1100 лет** трудовой деятельности. Ряд значений делится пополам на значении -3,3 года, а это означает, что большая часть значений со знаком '-', а среднее значение на уровне 156 лет. Появление такого массива данных тоже можно назвать системной ошибкой. Большое количество значений со знаком минус, значит, происходит процесс вычитания. Скорее всего, данные получены за счет не корректного вычитанния даты начала работы, например в EXCEL.
4. Были выявлены следующие артефакты:
   * Информация в столбце *'children'* минимальное значение равно '-1', а максимальное значение равно '20'. Таких значений быть не может исходя из анализа процентиля.Были внесены изменения в данные. Количество детей у заемщиков может варьироваться от 0 до 5.
   * В столбце *'dob_years'* минимальное значение возраста является '0'. Такого значения быть не может. Трудоспособный возраст начинается с 18 лет. В датасете возраст 18 лет был упущен. Значение '0' заменено на '18'.
   * В столбце 'gender' выявлено значение 'xna' в единственном экземпляре. Заменили на значение 'm'.
   * В столбце 'income_type' объединили 'компаньон' и 'предприниматель', так как компаньон является совладельцем компании. 
5. После проведения замены типа данных с численными значениями столбцов получили уменьшение памяти на 70% ***(с 1.15 MB до 0,35 MB)***. Провели замену типа данных в столбцах с  **'object'** на **'category'**. Уменьшение занимаемой памяти составило 99% ***(с 12.34 MB до 0,11 MB)***. В итоге уменьшение объема занимаемой памяти ***составило 97%*** без потери важной информации для работы.
6. Обработка методом drop_duplicates().reset_index позволила убрать 71 дублирующую строчку. Дублирующие строчки появляются при внесении в базу заемщиков, либо полное совпадение из-за отсутствия уникальных идентификаторов. Для поиска дубликатов были изменены заглавные буквы на строчные.
7. При помощи лемматизации **уменьшили количество целей получения кредита с 38 до 5**: свадьба, недвижимость, жилье, образование, автомобиль. При вводе данных в базу можно организовать систему с меньшим количеством целей для кредита.
8. **Заемщики, не имеющие детей**, выплачивают кредиты более дисциплинировано, чем заемщики имеющие 1 или 2 детей.
9. **Вдовы/вдовцы или заемщики в разводе** лучше выплачивают кредиты, чем не женатые/не замужние, либо живущие в гражданском браке.
10. **Заемщики, имеющие ежемесячный доход более 200 000**, выплачивают лучше, чем заемщики из других категорий, чем заемщики с доходом от 100 000 до 200 000.
11. **Ипотека,особенно связанная с покупкой жилого помещения**, выплачивается лучше, чем кредиты на образование и покупку автомобиля.
12. Можно сделать вывод, что **одинокие люди (были в браке) и без детей** лучше выплачивают кредит, особенно, если кредит на покупку жилого помещения, чем другие категории заемщиков. 
13. Данные показывают, что заемщики с доходом более 200 000 рублей более дисциплинированы в выплате кредитов, но тут можно предположить, что доход покрывает выплату кредита и не доставляет проблем заемщику. Можно было бы расширить уровень исследований, если бы были данные о стоимости взятого кредита и ежемесячного платежа по кредиту. 